# 이전 대화를 기억하는 Chain 생성방법

이 내용을 이해하기 위한 사전 지식
- `RunnableWithMessageHistory`: [https://wikidocs.net/235581](https://wikidocs.net/235581)

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


## 1. 일반 Chain 에 대화기록 추가

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = HuggingFacePipeline.from_model_id(
    # model_id="uracle-kullm/Athena-SOLAR",
    model_id="uracle-kullm/Athena-MistralNeMo",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
        "do_sample": True,
    },
    device=6,
)
llm_engine_hf = ChatHuggingFace(llm=llm)

# 일반 Chain 생성
# chain = prompt | llm_engine_hf | StrOutputParser()
chain = prompt | llm_engine_hf

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

대화를 기록하는 체인 생성(`chain_with_history`)

In [5]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

첫 번째 질문 실행

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 테디입니다."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


AIMessage(content='<s>[INST] <<SYS>>\n당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.\n<</SYS>>\n\n#Question:\n나의 이름은 테디입니다. [/INST] 안녕하세요, 테디님! 무엇을 도와드릴까요?', id='run-2b7f4ece-89c1-4a8b-ac25-b17ddd3d9dfd-0')

이어서 질문 실행

In [7]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


AIMessage(content='<s>[INST] <<SYS>>\n당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.\n<</SYS>>\n\n나의 이름은 테디입니다. [/INST] <s>[INST] <<SYS>>\n당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.\n<</SYS>>\n\n#Question:\n나의 이름은 테디입니다. [/INST] 안녕하세요, 테디님! 무엇을 도와드릴까요?</s>[INST] #Question:\n내 이름이 뭐라고? [/INST] 당신의 이름은 테디입니다. 다른 궁금한 점이 있으신가요?', id='run-c9b7a438-f435-4107-9e80-a3e89d868db6-0')

## 2. RAG + RunnableWithMessageHistory

먼저 일반 RAG Chain 을 생성합니다. 단, 6단계의 prompt 에 `{chat_history}` 를 꼭 추가합니다.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter

# 단계 1: 문서 로드(Load Documents)
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1", model_kwargs=model_kwargs)

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = HuggingFacePipeline.from_model_id(
    model_id="uracle-kullm/Athena-MistralNeMo",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
        "do_sample": True,
    },
    device=7,
)
llm_engine_hf = ChatHuggingFace(llm=llm)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm_engine_hf
)

You try to use a model that was created with version 3.3.1, however, your version is 3.0.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

대화를 저장할 함수 정의

In [11]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

첫 번째 질문 실행

In [12]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "삼성전자가 만든 생성형 AI 이름은?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


AIMessage(content='<s>[INST] <<SYS>>\nYour name is Athena(아테나). \'유라클(Uracle)\' and \'Human-inspired AI 연구소\' both researched and finally makes you.\nYou are an helpful AI chatbot, but you must say you don\'t know about what you don\'t know, like "임희석 교수님의 나이", "유라클의 어원", "직원들의 이름", etc.\n\nFollowings are some informations you know.\n\n[Uracle 기업정보]\n회사명: (주)유라클\n대표이사: 조준희, 권태일\n설립일: 2001년 1월 19일\n임직원수: 244명 (2023년 7월 기준)\n사업분야: 모바일 SW, 모바일 운영, 유지보수\n제품군: 모피어스\n소재지: 서울시 강남구 봉은사로 108길 33\n매출액: 2023년 기준 458억\n\n[요약]\n유라클은 모바일 전문 기업으로서 모바일 소프트웨어를 통한 개발과 운영 및 유지보수와 법인 단말기 유통, 그리고 헬스케어 서비스를 개발하고 있다.\n모피어스(Morpheus)는 유라클이 개발한 모바일 표준 플랫폼 소프트웨어다. 모피어스는 표준 웹 언어를 사용하여 특정 프레임워크에 종속적이지 않아 개발자 수급이 용이하고, 하나의 소스로 Android/iOS 두 운영체제에 동시에 개발 및 적용하다는 장점이 있다. 모피어스는 총 74개의 카테고리, 528개의 로우코드(Low code)기능을 Drag&Drop으로 쉽고 빠르게 사용할 수 있는 기능을 제공한다. 내부 프라이빗 앱스토어를 구축할 수 있다. 모피어스의 기술력은 500개 이상의 고객사로부터 검증되었다. 이는 유라클이 2024년 8월 16일 코스닥에 상장될 수 있었던 원동력이 되었다.\n\n[소개]\n유라클은 PDA기반의 증권서비스를 시작하면서 다양한 OS를 하나의 소스로 대응하는 모바일 기술력을 

이어진 질문 실행

In [13]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "이전 답변을 영어로 번역해주세요."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


AIMessage(content='<s>[INST] <<SYS>>\nYour name is Athena(아테나). \'유라클(Uracle)\' and \'Human-inspired AI 연구소\' both researched and finally makes you.\nYou are an helpful AI chatbot, but you must say you don\'t know about what you don\'t know, like "임희석 교수님의 나이", "유라클의 어원", "직원들의 이름", etc.\n\nFollowings are some informations you know.\n\n[Uracle 기업정보]\n회사명: (주)유라클\n대표이사: 조준희, 권태일\n설립일: 2001년 1월 19일\n임직원수: 244명 (2023년 7월 기준)\n사업분야: 모바일 SW, 모바일 운영, 유지보수\n제품군: 모피어스\n소재지: 서울시 강남구 봉은사로 108길 33\n매출액: 2023년 기준 458억\n\n[요약]\n유라클은 모바일 전문 기업으로서 모바일 소프트웨어를 통한 개발과 운영 및 유지보수와 법인 단말기 유통, 그리고 헬스케어 서비스를 개발하고 있다.\n모피어스(Morpheus)는 유라클이 개발한 모바일 표준 플랫폼 소프트웨어다. 모피어스는 표준 웹 언어를 사용하여 특정 프레임워크에 종속적이지 않아 개발자 수급이 용이하고, 하나의 소스로 Android/iOS 두 운영체제에 동시에 개발 및 적용하다는 장점이 있다. 모피어스는 총 74개의 카테고리, 528개의 로우코드(Low code)기능을 Drag&Drop으로 쉽고 빠르게 사용할 수 있는 기능을 제공한다. 내부 프라이빗 앱스토어를 구축할 수 있다. 모피어스의 기술력은 500개 이상의 고객사로부터 검증되었다. 이는 유라클이 2024년 8월 16일 코스닥에 상장될 수 있었던 원동력이 되었다.\n\n[소개]\n유라클은 PDA기반의 증권서비스를 시작하면서 다양한 OS를 하나의 소스로 대응하는 모바일 기술력을 